# Price Analysis of Reference GPUs


## Data collected 10/19/21


In [52]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from plotnine import ggplot, aes, geom_sina, geom_point, geom_violin, coord_flip, coord_cartesian, xlim

In [2]:
cwd = os.path.abspath('C:/Users/2018Wei/OneDrive - University of North Carolina at Chapel Hill/gpu-prices/data')
os.chdir(cwd)
print(os.getcwd())

C:\Users\2018Wei\OneDrive - University of North Carolina at Chapel Hill\gpu-prices\data


In [3]:
files = os.listdir(cwd)

**Since every gpu has its own excel file, loop through every excel file to create one dataframe**

In [4]:
df_total = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        excel_file = pd.ExcelFile(file)
        sheets = excel_file.sheet_names
        for sheet in sheets:               # loop through sheets inside an Excel file
            df = excel_file.parse(sheet_name = sheet)
            df_total = df_total.append(df)

In [5]:
df_total.card.unique()

array(['amd 6700 xt', 'amd 6800', 'amd 6800 xt', 'amd 6900 xt',
       'gtx 3060 ti', 'gtx 3070', 'gtx 3070 ti', 'gtx 3080',
       'gtx 3080 ti', 'gtx 3090'], dtype=object)

**For this project, we only need columns card, date sold, and price so drop the rest**

In [6]:
df_total.head()

,card,product title,product url,subtitle1,subtitle2,date sold,price
0,amd 6700 xt,New ListingAMD Radeon RX 6700 XT 12GB GDDR6 Gr...,https://www.ebay.com/itm/334186500440?epid=504...,Brand New · AMD · AMD Radeon RX 6700 XT · 12 GB,NaN,"Oct 19, 2021",$900.00
1,amd 6700 xt,New ListingAMD Radeon RX 6700 XT 12GB Graphics...,https://www.ebay.com/itm/224652615904?epid=504...,Brand New · AMD · AMD Radeon RX 6700 XT · 12 GB,NaN,"Oct 19, 2021",$825.00
2,amd 6700 xt,New ListingAMD Radeon RX 6700 XT 12GB GDDR6 Gr...,https://www.ebay.com/itm/313717565151?epid=504...,Brand New · AMD · AMD Radeon RX 6700 XT · 12 GB,NaN,"Oct 19, 2021",$849.99
3,amd 6700 xt,New ListingAMD Radeon RX 6700 XT 12GB GDDR6 Gr...,https://www.ebay.com/itm/224652871378?epid=160...,Brand New · AMD · AMD Radeon RX 6700 XT · 12 GB,NaN,"Oct 19, 2021",$850.00
4,amd 6700 xt,AMD Radeon RX 6700 XT 12GB GDDR6 Graphics Card...,https://www.ebay.com/itm/334186784072?epid=160...,Brand New · AMD · AMD Radeon RX 6700 XT · 12 GB,NaN,"Oct 18, 2021",$850.00


In [7]:
gpu_df = df_total.drop(columns = ['product title', 'product url', 'subtitle1', 'subtitle2'])

In [8]:
gpu_df

,card,date sold,price
0,amd 6700 xt,"Oct 19, 2021",$900.00
1,amd 6700 xt,"Oct 19, 2021",$825.00
2,amd 6700 xt,"Oct 19, 2021",$849.99
3,amd 6700 xt,"Oct 19, 2021",$850.00
4,amd 6700 xt,"Oct 18, 2021",$850.00
...,...,...,...
1395,gtx 3090,"Jul 22, 2021","$2,100.00"
1396,gtx 3090,"Jul 21, 2021","$2,100.00"
1397,gtx 3090,"Jul 21, 2021","$1,999.00"
1398,gtx 3090,"Jul 21, 2021","$2,089.99"


**After obtaining our final set of columns, want to make sure that each column is the right data type. We wannt date sold to be datetime and price to be type float. The column card is fine as a string.**


gpu.dtypes

In [9]:
gpu_df = gpu_df.convert_dtypes()

In [10]:
gpu_df.dtypes

card         string
date sold    string
price        string
dtype: object

In [11]:
gpu_df[gpu_df['card'] == 'amd 6700 xt']

,card,date sold,price
0,amd 6700 xt,"Oct 19, 2021",$900.00
1,amd 6700 xt,"Oct 19, 2021",$825.00
2,amd 6700 xt,"Oct 19, 2021",$849.99
3,amd 6700 xt,"Oct 19, 2021",$850.00
4,amd 6700 xt,"Oct 18, 2021",$850.00
...,...,...,...
940,amd 6700 xt,"Jul 22, 2021",$699.99
941,amd 6700 xt,"Jul 22, 2021",$749.98
942,amd 6700 xt,"Jul 22, 2021",$839.50
943,amd 6700 xt,"Jul 22, 2021",$700.00


In [12]:
gpu_df['price'] = gpu_df['price'].str.replace(',', '').str.replace('$', '').astype(float)

<ipython-input-12-9344ef306cce>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [13]:
gpu_df['price']

0        900.00
1        825.00
2        849.99
3        850.00
4        850.00
         ...   
1395    2100.00
1396    2100.00
1397    1999.00
1398    2089.99
1399    2076.00
Name: price, Length: 7376, dtype: float64

In [14]:
gpu_df.dtypes

card          string
date sold     string
price        float64
dtype: object

In [15]:
gpu_df['date sold'] = pd.to_datetime(df['date sold'])

In [16]:
gpu_df.dtypes

card                 string
date sold    datetime64[ns]
price               float64
dtype: object

**Now we look for outliers. There are some obviously not real sale prices, such as a gtxo 3060 ti for a dollar, so I want to remove those as they add nothing to my analysis**


In [23]:
gpu_df

,card,date sold,price
0,amd 6700 xt,2021-10-19,900.00
1,amd 6700 xt,2021-10-19,825.00
2,amd 6700 xt,2021-10-19,849.99
3,amd 6700 xt,2021-10-19,850.00
4,amd 6700 xt,2021-10-19,850.00
...,...,...,...
1395,gtx 3090,2021-07-22,2100.00
1396,gtx 3090,2021-07-21,2100.00
1397,gtx 3090,2021-07-21,1999.00
1398,gtx 3090,2021-07-21,2089.99


In [57]:
gpu_df.to_csv('gpus.csv')